In [49]:
import sys
sys.path.append("..")

from services.segments_database import do_query
import pandas as pd
import re
from sklearn.metrics import confusion_matrix, classification_report


# Define Patterns

In [2]:
patterns = [
    r"\szk[0-9]*",
    r"zuschlag\w*"
]

# Code

## get data

In [4]:
query = '''
    SELECT *, 
    IF(length(GROUP_CONCAT(word.word SEPARATOR ' ')) > 0, GROUP_CONCAT(word.word SEPARATOR ' '), '')  as words 
    FROM trainingdata 
    LEFT JOIN word ON trainingdata.id = word.chapter_id
    WHERE (label = 'yes' 
    OR label = 'no')
    and round = 1000
    GROUP BY trainingdata.id
'''

In [5]:
res = do_query(query)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
print(df.shape)

(1000, 33)


In [6]:
df.head()

,id,text,chapter_idx,chapter_number,header,header_preprocessed,parent_header,parent_preprocessed,grandparent_header,grandparent_preprocessed,...,round,suggested_label,createdAt,updatedAt,id,word,tfidf,score,chapter_id,words
0,101,\nWeitere spezifische Grundlagen sind vorhande...,26,5.2,Spezifische Grundlagen\n,spezifisch grundlage,Grundlagen\n,grundlage,,,...,1000,0,2019-05-13 10:22:07,2019-05-13 12:28:07,2074.0,ausarbeitung,0.385710,471.0,101.0,ausarbeitung generalplanerangebotes grundlage ...
1,251,\nFolgende Schritte erfolgen bis zum Zuschlags...,26,7.1,Evaluationsphasen\n,evaluationsphase,Evaluation\n,evaluation,,,...,1000,0,2019-05-13 10:22:08,2019-05-13 12:28:10,4544.0,10,0.136392,2510.0,251.0,10 2 3 4 5 6 7 aktivität angebot ausschreibung...
2,509,\n2.6.1 \nAuf das vorliegende Vergabeverfahren...,36,2.6,Ausschreibungsbedingungen\n,ausschreibungsbedingung,,,,,...,1000,0,2019-05-13 10:22:04,2019-05-13 12:28:17,NaN,None,NaN,NaN,NaN,
3,779,\n3.1.1 \n.110 02 Schweizerische Südostbahn AG...,7,3.1,"Bauherr, Besteller, Eigentümer.\n",bauherr besteller eigentümer,"100\tORGANISATION BAUHERR, LAGE, ZWECKBESTIMMU...",100 organisation bauherr lage zweckbestimmung,,,...,1000,0,2019-05-13 10:22:05,2019-05-13 12:28:19,NaN,None,NaN,NaN,NaN,
4,800,\nDie Gasleitung wird im Bereich der Zentralst...,55,55,Gas\n,gas,,,,,...,1000,0,2019-05-13 10:22:03,2019-05-13 12:28:21,13165.0,110,0.253070,790.0,800.0,110 380 bereich erstellen gasleitung grabenlän...


## do matching

In [43]:
labels = []

for i, row in df.iterrows():
    match = False
    for pattern in patterns:
        if re.search(pattern, row["words"]) or re.search(pattern, row["header_preprocessed"]):
            match = True
    labels.append(match)

prediction = pd.concat([df, pd.Series(labels, name='pred')], axis=1)
prediction.head()

,id,text,chapter_idx,chapter_number,header,header_preprocessed,parent_header,parent_preprocessed,grandparent_header,grandparent_preprocessed,...,suggested_label,createdAt,updatedAt,id,word,tfidf,score,chapter_id,words,pred
0,101,\nWeitere spezifische Grundlagen sind vorhande...,26,5.2,Spezifische Grundlagen\n,spezifisch grundlage,Grundlagen\n,grundlage,,,...,0,2019-05-13 10:22:07,2019-05-13 12:28:07,2074.0,ausarbeitung,0.385710,471.0,101.0,ausarbeitung generalplanerangebotes grundlage ...,False
1,251,\nFolgende Schritte erfolgen bis zum Zuschlags...,26,7.1,Evaluationsphasen\n,evaluationsphase,Evaluation\n,evaluation,,,...,0,2019-05-13 10:22:08,2019-05-13 12:28:10,4544.0,10,0.136392,2510.0,251.0,10 2 3 4 5 6 7 aktivität angebot ausschreibung...,True
2,509,\n2.6.1 \nAuf das vorliegende Vergabeverfahren...,36,2.6,Ausschreibungsbedingungen\n,ausschreibungsbedingung,,,,,...,0,2019-05-13 10:22:04,2019-05-13 12:28:17,NaN,None,NaN,NaN,NaN,,False
3,779,\n3.1.1 \n.110 02 Schweizerische Südostbahn AG...,7,3.1,"Bauherr, Besteller, Eigentümer.\n",bauherr besteller eigentümer,"100\tORGANISATION BAUHERR, LAGE, ZWECKBESTIMMU...",100 organisation bauherr lage zweckbestimmung,,,...,0,2019-05-13 10:22:05,2019-05-13 12:28:19,NaN,None,NaN,NaN,NaN,,False
4,800,\nDie Gasleitung wird im Bereich der Zentralst...,55,55,Gas\n,gas,,,,,...,0,2019-05-13 10:22:03,2019-05-13 12:28:21,13165.0,110,0.253070,790.0,800.0,110 380 bereich erstellen gasleitung grabenlän...,False


## evaluation

In [44]:
# convert
prediction.loc[prediction['label'] == 'yes', 'label'] = 1
prediction.loc[prediction['label'] == 'no', 'label'] = 0
prediction.loc[prediction['pred'] == True, 'pred'] = 1
prediction.loc[prediction['pred'] == False, 'pred'] = 0

In [45]:
confusion_type = [];
for index, row in prediction.iterrows():
    if row.label == 0 and row.pred == 0:
        confusion_type.append('TN')
    elif row.label == 1 and row.pred == 1: 
        confusion_type.append('TP')
    elif row.label == 0 and row.pred == 1: 
        confusion_type.append('FP')
    elif row.label == 1 and row.pred == 0: 
        confusion_type.append('FN')
    else: 
        confusion_type.append('no match')
prediction = pd.concat([prediction, pd.Series(confusion_type, name='confusion_type')], axis=1)
prediction.head()

,id,text,chapter_idx,chapter_number,header,header_preprocessed,parent_header,parent_preprocessed,grandparent_header,grandparent_preprocessed,...,createdAt,updatedAt,id,word,tfidf,score,chapter_id,words,pred,confusion_type
0,101,\nWeitere spezifische Grundlagen sind vorhande...,26,5.2,Spezifische Grundlagen\n,spezifisch grundlage,Grundlagen\n,grundlage,,,...,2019-05-13 10:22:07,2019-05-13 12:28:07,2074.0,ausarbeitung,0.385710,471.0,101.0,ausarbeitung generalplanerangebotes grundlage ...,0,TN
1,251,\nFolgende Schritte erfolgen bis zum Zuschlags...,26,7.1,Evaluationsphasen\n,evaluationsphase,Evaluation\n,evaluation,,,...,2019-05-13 10:22:08,2019-05-13 12:28:10,4544.0,10,0.136392,2510.0,251.0,10 2 3 4 5 6 7 aktivität angebot ausschreibung...,1,FP
2,509,\n2.6.1 \nAuf das vorliegende Vergabeverfahren...,36,2.6,Ausschreibungsbedingungen\n,ausschreibungsbedingung,,,,,...,2019-05-13 10:22:04,2019-05-13 12:28:17,NaN,None,NaN,NaN,NaN,,0,FN
3,779,\n3.1.1 \n.110 02 Schweizerische Südostbahn AG...,7,3.1,"Bauherr, Besteller, Eigentümer.\n",bauherr besteller eigentümer,"100\tORGANISATION BAUHERR, LAGE, ZWECKBESTIMMU...",100 organisation bauherr lage zweckbestimmung,,,...,2019-05-13 10:22:05,2019-05-13 12:28:19,NaN,None,NaN,NaN,NaN,,0,TN
4,800,\nDie Gasleitung wird im Bereich der Zentralst...,55,55,Gas\n,gas,,,,,...,2019-05-13 10:22:03,2019-05-13 12:28:21,13165.0,110,0.253070,790.0,800.0,110 380 bereich erstellen gasleitung grabenlän...,0,TN


In [50]:
print(classification_report(prediction['label'], prediction['pred']))
print(confusion_matrix(prediction['label'], prediction['pred']))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       969
           1       0.28      0.52      0.36        31

   micro avg       0.94      0.94      0.94      1000
   macro avg       0.63      0.74      0.66      1000
weighted avg       0.96      0.94      0.95      1000

[[927  42]
 [ 15  16]]
